# WCA Results - Data Preparation

Created by Michael George (AKA Logiqx)

Extract data from the WCA database and reformat into individual CSV files for each event.

Note: This script can only be run if you have the contents of the "private" data folder.

## Database Parameters

Connection details for MySQL / MariaDB database

In [1]:
# Use the OS library to read environment variables
import os

# You will need to update the password in $HOME/.my.cnf
hostname = os.environ['MYSQL_HOSTNAME']
database = os.environ['MYSQL_DATABASE']
username = os.environ['MYSQL_USER']

# Path of private data folder
projdir = '..'

## Generic SQL Function

Simple function to run a SQL script

In [2]:
import sqlparse, pymysql
import csv

def runSqlScript(source, extract = False):
    with open(source) as infile:
        script = infile.read()
        statements = sqlparse.split(script)

        con = pymysql.connect(host=hostname, user=username, db=database,
                              read_default_file='~/.my.cnf', autocommit=True)
        with con:
            cur = con.cursor()

            for statement in statements:
                if statement:
                    cur.execute(statement)

                    if extract:
                        rows = cur.fetchall()

                        fn = os.path.join(projdir, 'data', 'private', 'extract', rows[0][0] + '.csv')
                        with open(fn, 'w') as outfile:
                            csvWriter = csv.writer(outfile, quoting = csv.QUOTE_MINIMAL, lineterminator = os.linesep)

                            for row in rows:
                                csvWriter.writerow(row[1:])

## Run Extracts

Extract data from database for subsequent analysis - percentiles, rankings, etc

In [3]:
# Remove previous extracts
import os, glob
for f in glob.glob(os.path.join(projdir, 'data', 'private', 'extract', '*.csv')):
    os.remove(f)

# Create synthetic results, etc
runSqlScript(os.path.join(projdir, 'sql', 'create_senior_averages.sql'))
runSqlScript(os.path.join(projdir, 'sql', 'create_senior_singles.sql'))

# Ensure name and country is up-to-date
runSqlScript(os.path.join(projdir, 'sql', 'apply_persons.sql'))

# Extract seniors
runSqlScript(os.path.join(projdir, 'sql', 'extract_senior_details.sql'), extract = True)
runSqlScript(os.path.join(projdir, 'sql', 'extract_senior_averages.sql'), extract = True)
runSqlScript(os.path.join(projdir, 'sql', 'extract_senior_singles.sql'), extract = True)

# Extract WCA
runSqlScript(os.path.join(projdir, 'sql', 'extract_wca_aggs.sql'), extract = True)

# Extract future comps
runSqlScript(os.path.join(projdir, 'sql', 'extract_future_competitions.sql'), extract = True)

# Extract WCA lookups
runSqlScript(os.path.join(projdir, 'sql', 'extract_wca_lookups.sql'), extract = True)

## Generic Processing

Turn the raw database extracts into a standard format:
* Split into multiple files - one file per event
* Standardise the layout - CSV with minimal quoting
* Apply time limits / cutoffs to aggregated data

Note: All of the output files can be made public due to the application of time limits / cutoffs

In [4]:
import os, csv

from EventsLib import *

def writeResults(basename, event, eventResults):
    """Write event results from memory to CSV"""

    fn = os.path.join(projdir, 'data', 'public', basename, event + '.csv')
    with open(fn, 'w') as f:
        csvWriter = csv.writer(f, quoting = csv.QUOTE_MINIMAL, lineterminator = os.linesep)
        for eventResult in eventResults:
            csvWriter.writerow(eventResult)


def prepareCounts(basename, subfolder):
    """Split file into individual events and apply time limits"""

    # Ensure that a CSV exists for all events, even if empty
    for event in events:
        writeResults(basename, event[0], [])
        
    fn = os.path.join(projdir, 'data', subfolder, basename + '.csv')
    with open(fn, 'r') as f:
        csvReader = csv.reader(f)
        
        # Initilise event
        event = None
        cut1 = 0
        cut2 = 0
        cut3 = 0
        eventResults = []

        # Initilise result
        result = None
        count = 0

        # Process each row individually
        for inputRow in csvReader:

            # Only process the current row if it is a recognised event
            if inputRow[0] == event or inputRow[0] in eventsDict:

                thisEvent = inputRow[0]
                thisResult = int(inputRow[1])

                # Detect change of event
                if thisEvent != event:
                    # Buffer the final result
                    if count != 0:
                        eventResults.append([result, count])
                    # Save the previous event
                    if event:
                        writeResults(basename, event, eventResults)

                    # Initilise event
                    event = thisEvent
                    cut1 = int(eventsDict[event][3])
                    cut2 = int(eventsDict[event][4])
                    cut3 = int(eventsDict[event][5])
                    eventResults = []

                    # Initilise result
                    result = None
                    count = 0
                    
                # Apply cutoffs
                if thisResult > cut3:
                    thisResult = cut3
                elif thisResult > cut2:
                    thisResult = thisResult // 60 * 60
                elif thisResult > cut1:
                    thisResult = thisResult // 10 * 10

                # Detect change of result
                if thisResult != result:
                    # Buffer the current result
                    if count != 0:
                        eventResults.append([result, count])

                    result = thisResult
                    count = 0

                count += int(inputRow[2])

        # Save the final event
        if count != 0:
            eventResults.append([result, count])
        writeResults(basename, event, eventResults)

        
def prepareResults(basename, subfolder):
    """Split file into individual events"""
    
    fn = os.path.join(projdir, 'data', subfolder, basename + '.csv')
    with open(fn, 'r') as f:
        csvReader = csv.reader(f)
        
        event = None
        eventResults = []

        # Process each row individually
        for inputRow in csvReader:

            # Only process the current row if it is a recognised event
            if inputRow[0] == event or inputRow[0] in eventsDict:

                # Detect change of event
                if inputRow[0] != event:
                    if event:
                        writeResults(basename, event, eventResults)
                    event = inputRow[0]
                    eventResults = []

                # Buffer the current result
                eventResults.append(inputRow[1:])

        # Save the final event
        writeResults(basename, event, eventResults)


def prepareSimple(basename, subfolder):
    """Essentially a file copy but it will reformat the records if necessary"""
    
    rows = []
    
    # Read rows using the CSV reader
    fn = os.path.join(projdir, 'data', subfolder, basename + '.csv')
    with open(fn, 'r') as f:
        csvReader = csv.reader(f)
        for inputRow in csvReader:
            rows.append(inputRow)

    # Write rows using the CSV writer
    fn = os.path.join(projdir, 'data', 'public', basename + '.csv')
    with open(fn, 'w') as f:
        csvWriter = csv.writer(f, quoting = csv.QUOTE_MINIMAL, lineterminator = os.linesep)
        for row in rows:
            csvWriter.writerow(row)

## Format Extracts

Prepare all of the CSV files

In [5]:
extract_dir = os.path.join('private', 'extract')

# Process known seniors from local database export
prepareSimple('known_senior_details', extract_dir)
prepareResults('known_senior_averages', extract_dir)
prepareResults('known_senior_singles', extract_dir)

# Process indicative seniors from local database export
prepareResults('senior_averages', extract_dir)
prepareCounts('senior_averages_agg', extract_dir)

# Process WCA from local database export
prepareCounts('wca_averages_agg', extract_dir)
prepareCounts('wca_singles_agg', extract_dir)

# Process future competitions
prepareSimple('future_competitions', extract_dir)

# Process WCA lookups
prepareSimple('wca_lookup_events', extract_dir)
prepareSimple('wca_lookup_countries', extract_dir)
prepareSimple('wca_lookup_continents', extract_dir)

print('Extracts formatted!')

Extracts formatted!


## All Done!